<a href="https://colab.research.google.com/github/aminojagh/fast-ai/blob/main/NB8-Stable_Diffusion_DeepDive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Deep Dive

Stable Diffusion is a powerful text-to-image model. There are various websites and tools to make using it as easy as possible. It is also [integrated into the Huggingface diffusers library](https://huggingface.co/blog/stable_diffusion) where generating images can be as simple as:
```python
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained(
  "CompVis/stable-diffusion-v1-4",
  variant="fp16",
  torch_dtype=torch.float16,
  use_auth_token=True
).to("cuda")
image = pipe("An astronaught scuba diving").images[0]

```

In this notebook we're going to dig into the code behind these easy-to-use interfaces, to see what is going on under the hood.

## Setup & Imports

In [ ]:
# !pip install -q --upgrade transformers diffusers ftfy accelerate

In [ ]:
from pathlib import Path
from huggingface_hub import notebook_login
if not (Path.home()/'.cache/huggingface'/'token').exists():
  notebook_login()

In [ ]:
from base64 import b64encode
import numpy, torch, os
from IPython.display import HTML
from matplotlib import pyplot as plt
from PIL import Image
from tqdm.auto import tqdm

from torch import autocast
from torchvision import transforms as tfms
from transformers import CLIPTextModel, CLIPTokenizer, logging
from diffusers import (AutoencoderKL,
                       LMSDiscreteScheduler,
                       UNet2DConditionModel)

In [ ]:
torch.manual_seed(1)

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
if torch.cuda.is_available():
  torch_device = "cuda"
elif torch.backends.mps.is_available():
  torch_device = "mps"
  os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"
else: torch_device = "cpu"

This code (and that in the next section) comes from the [Huggingface example notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb).

This will download and set up the relevant models and components we'll be using.

If you've loaded a pipeline, you can also access these components using `pipe.unet`, `pipe.vae` and so on.

In this notebook we aren't doing any memory-saving tricks.

if you find yourself running out of GPU RAM, look at the pipeline code for inspiration with things like
- attention slicing
- switching to half precision (fp16)
- keeping the VAE on the CPU and other modifications.

## Loading the models

In [ ]:
# Load the autoencoder model which will be used
# to decode the latents into image space.
vae = AutoencoderKL.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    subfolder="vae"
)

# The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    subfolder="unet"
)

# Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer\
              .from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel\
              .from_pretrained("openai/clip-vit-large-patch14")

# The noise scheduler
scheduler = LMSDiscreteScheduler(
    beta_start=0.00085,beta_end=0.012,
    beta_schedule="scaled_linear",
    num_train_timesteps=1000
)

def set_timesteps(scheduler, num_inference_steps):
    scheduler.set_timesteps(num_inference_steps)
    # minor fix to ensure MPS compatibility, fixed in diffusers PR 3925
    scheduler.timesteps = scheduler.timesteps.to(torch.float32)

# To the GPU we go!
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device);

## A diffusion loop

What we want to do in this notebook is dig a little deeper into how **latent stable diffusion** works, so we'll start by checking that the example code runs. Again, this is adapted from the [HF notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb) and looks very similar to what you'll find if you inspect [the `__call__()` method of the stable diffusion pipeline](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py#L200).  

In [ ]:
# Some settings
prompt = ["A watercolor painting of an otter"]

# default height of Stable Diffusion
height = 512

# default width of Stable Diffusion
width = 512

# Number of denoising steps
num_inference_steps = 30

# Scale for classifier-free guidance
guidance_scale = 7.5

# Seed generator to create the inital latent noise
generator = torch.manual_seed(32)
batch_size = 1

In [ ]:
# Prep text
def create_text_emb(prompt):
  text_input = tokenizer(
      prompt,
      padding="max_length",
      max_length=tokenizer.model_max_length,
      truncation=True,
      return_tensors="pt"
  )
  with torch.no_grad():
      text_embeddings = text_encoder(
          text_input.input_ids.to(torch_device)
      ).get('last_hidden_state')

  uncond_input = tokenizer(
      [""] * batch_size,
      padding="max_length",
      max_length=text_input.input_ids.shape[-1],
      return_tensors="pt"
  )
  with torch.no_grad():
      uncond_embeddings = text_encoder(
          uncond_input.input_ids.to(torch_device)
      ).get('last_hidden_state')


  text_embeddings = torch.cat(
      [
          uncond_embeddings,
          text_embeddings
      ]
  )
  return text_embeddings

text_embeddings = create_text_emb(prompt)

In [ ]:
# Prep Scheduler
set_timesteps(scheduler,num_inference_steps)

# Prep latents
latents = torch.randn(
  (batch_size, unet.config.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents.to(torch_device)
# Scaling (previous versions did latents=latents*self.scheduler.sigmas[0]
latents = latents * scheduler.init_noise_sigma

In [ ]:
# Loop
with autocast("cuda"):
    # will fallback to CPU if no CUDA; no autocast for MPS
    for i, t in tqdm(
        enumerate(scheduler.timesteps),
        total=len(scheduler.timesteps)
    ):
        # expand the latents if we are doing classifier-free
        # guidanceto avoid doing two forward passes.
        latent_model_input = torch.cat([latents] * 2)
        sigma = scheduler.sigmas[i]
        # Scale the latents (preconditioning):
        latent_model_input = scheduler\
                              .scale_model_input(latent_model_input, t)
        # for Diffusers 0.3 and below
        # latent_model_input = latent_model_input/((sigma**2 + 1) ** 0.5)

        # predict the noise residual
        with torch.no_grad():
            noise_pred = unet(
                latent_model_input,
                timestep = t,
                encoder_hidden_states=text_embeddings
            ).sample

        # perform guidance
        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        noise_pred = (
            noise_pred_uncond
            + guidance_scale * (noise_pred_text - noise_pred_uncond)
        )

        # compute the previous noisy sample x_t -> x_t-1
        latents = scheduler.step(noise_pred, t, latents).prev_sample
        # for Diffusers 0.3 and below
        # latents = scheduler.step(noise_pred, i, latents)["prev_sample"]

In [ ]:
# scale and decode the image latents with vae
latents = 1 / 0.18215 * latents
with torch.no_grad():
    image = vae.decode(latents).sample

# Display
image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]

It's working, but that's quite a bit of code! Let's look at the components one by one.

## The Autoencoder (AE)

The AE can 'encode' an image into some sort of latent representation, and decode this back into an image.

In [ ]:
def pil_to_latent(input_im):
    # Single image -> single latent in a batch (so size 1, 4, 64, 64)
    with torch.no_grad():
        latent = vae.encode(
            tfms.ToTensor()(input_im).unsqueeze(0).to(torch_device)*2-1
        ) # Note scaling
    return 0.18215 * latent.latent_dist.sample()

def latents_to_pil(latents):
    # bath of latents -> list of images
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images

We'll use a pic from the web here, but you can load your own instead by uploading it and editing the filename in the next cell.

In [ ]:
# Download a demo Image
!curl --output macaw.jpg 'https://lafeber.com/pet-birds/wp-content/uploads/2018/06/Scarlet-Macaw-2.jpg'

In [ ]:
# Load the image with PIL
input_image = Image.open('macaw.jpg').resize((512, 512))
print("original image")
display(input_image)

# Encode to the latent space
encoded = pil_to_latent(input_image)
print(f"encoded latent shape: {encoded.shape}")

# Let's visualize the four channels of this latent representation:
fig, axs = plt.subplots(1, 4, figsize=(16, 4))
for c in range(4):
    axs[c].imshow(encoded[0][c].cpu(), cmap='Greys')

# Decode this latent representation back into an image
decoded = latents_to_pil(encoded)[0]
print("decoded image")
display(decoded)

In this particular case the compression factor is 48, we start with a 3x512x512(chxhtxwd) image and it get compressed to a latent vector 4x64x64. Each 3x8x8 pixel volume in the input image gets compressed down to just 4 numbers(4x1x1). You can find AEs with a higher compression ratio (eg f16 like some popular VQGAN models) but at some point they begin to introduce artifacts that we don't want.


***Why do we even use an autoencoder?***
-  We can do diffusion in pixel space - where the model gets all the image data as inputs and produces an output prediction of the same shape. But this means processing a LOT of data, and make high-resolution generation very computationally expensive.

## The Scheduler

- During training, we add some noise to an image an then have the model try to predict the noise. If we always added a ton of noise, the model might not have much to work with. If we only add a tiny amount, the model won't be able to do much with the random starting points we use for sampling. So during training the amount is varied, according to some distribution.

- During sampling, we want to 'denoise' over a number of steps. How many steps and how much noise we should aim for at each step are going to affect the final result.

- The scheduler is in charge of handling all of these details. For example: `scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)` sets up a scheduler that matches the one used to train this model. When we want to sample over a smaller number of steps, we set this up with `scheduler.set_timesteps`:

In [ ]:
# Setting the number of sampling steps:
set_timesteps(scheduler, 15)
# See these in terms of the original 1000 steps used for training:
print(scheduler.timesteps)
# Look at the equivalent noise levels:
print(scheduler.sigmas)

During sampling, we'll start at a high noise level (in fact, our input will be pure noise) and gradually 'denoise' down to an image, according to this schedule.

In [ ]:
# Plotting this noise schedule:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].plot(scheduler.sigmas)
axs[0].set_title('Noise Schedule')
axs[0].set_xlabel('Sampling step')
axs[0].set_ylabel('sigma')

axs[1].plot(scheduler.timesteps)
axs[1].set_title('Time Steps')
axs[1].set_xlabel('Sampling step')
axs[1].set_ylabel('Time Step')

plt.show()

This 'sigma' is the amount of noise added to the latent representation. Let's visualize what this looks like by adding a bit of noise to our encoded image and then decoding this noised version:

In [ ]:
noise = torch.randn_like(encoded) # Random noise
sampling_step = 10
# Equivalent to step 10 out of 15 in the schedule above
# What does this look like at different timesteps?
# TODO: Experiment and see for yourself!
encoded_and_noised = scheduler.add_noise(
    encoded,
    noise,
    timesteps=torch.tensor([scheduler.timesteps[sampling_step]])
)
# for Diffusers 0.3 and below
# encoded_and_noised = scheduler.add_noise(encoded, noise, timestep)

latents_to_pil(encoded_and_noised.float())[0] # Display

you can run `??scheduler.add_noise` to see that `scheduler.add_noise` function literally just adds noise scaled by sigma: `noisy_samples = original_samples + noise * sigmas`

- Other diffusion models may be trained with different noising and scheduling approaches, some of which keep the variance fairly constant across noise levels ('variance preserving') with different scaling and mixing tricks instead of having noisy latents with higher and higher variance as more noise is added ('variance exploding').

- If we want to start from random noise instead of a noised image, we need to scale it by the largest sigma value used during training, ~14 in this case. And before these noisy latents are fed to the model they are scaled again in the so-called pre-conditioning step:
`latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)` (now handled by `latent_model_input = scheduler.scale_model_input(latent_model_input, t)`).
Again, this scaling/pre-conditioning differs between papers and implementations, so keep an eye out for this if you work with a different type of diffusion model.

## Loop starting from noised version of input (AKA image2image)

- Let's see what happens when we use our image as a starting point, adding some noise and then doing the final few denoising steps in the loop with a new prompt.

- We'll use a similar loop to the first demo, but we'll skip the first `start_step` steps.

- To noise our image we'll use code like that shown above, using the scheduler to noise it to a level equivalent to step 10 (`start_step`).

In [ ]:
prompt = ["A colorful dancer, nat geo photo"]
text_embeddings = create_text_emb(prompt)
# Number of denoising steps
num_inference_steps = 50
# Scale for classifier-free guidance
guidance_scale = 8
# Prep Scheduler (setting the number of inference steps)
set_timesteps(scheduler, num_inference_steps)
# Prep latents (noising appropriately for start_step)
start_step = 10
start_sigma = scheduler.sigmas[start_step]
noise = torch.randn_like(encoded)
latents = scheduler.add_noise(
    encoded,
    noise,
    timesteps=torch.tensor([scheduler.timesteps[start_step]])
).to(torch_device).float()

# Loop
for i, t in tqdm(
    enumerate(scheduler.timesteps),
    total=len(scheduler.timesteps)
):
    if i >= start_step:
    # << This is the only modification to the loop we do
        latent_model_input = torch.cat([latents] * 2)
        sigma = scheduler.sigmas[i]
        latent_model_input = scheduler\
                      .scale_model_input(latent_model_input, t)

        with torch.no_grad():
            noise_pred = unet(
                latent_model_input,
                timestep = t,
                encoder_hidden_states=text_embeddings
            ).sample
        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        noise_pred = (
            noise_pred_uncond
            + guidance_scale * (noise_pred_text - noise_pred_uncond)
        )
        latents = scheduler.step(noise_pred, t, latents).prev_sample

latents_to_pil(latents)[0]

to get further away from the input image.
- add more noise
- do more denoising steps

Explore changing how many steps are skipped and see how this affects the amount the image changes from the input.

## Exploring the text -> embedding pipeline

We use a text encoder model to turn our text into a set of 'embeddings' which are fed to the diffusion model as conditioning. Let's follow a piece of text through this process and see how it works.

In [ ]:
prompt = 'A picture of a puppy'
# Turn the text into a sequnce of tokens:
text_input = tokenizer(
    prompt,
    padding="max_length",
    max_length=tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
)
text_input['input_ids'][0] # View the tokens

# See the individual tokens
for t in text_input['input_ids'][0][:7]:
    print(t, tokenizer.decoder.get(int(t)))

# Grab the output embeddings
encoder_out = text_encoder(text_input.input_ids.to(torch_device))
print(encoder_out.keys())
output_embeddings = encoder_out['last_hidden_state']
print('Shape:', output_embeddings.shape)
output_embeddings

To get these input embeddings, there are actually two steps - as revealed by inspecting `text_encoder.text_model.embeddings`:

In [ ]:
text_encoder.text_model.embeddings

In [ ]:
# Access the embedding layer
token_emb_layer = text_encoder.text_model.embeddings.token_embedding
print("the token emb layer:")
display(token_emb_layer) # Vocab size 49408, emb_dim 768

token_id = 6829
print(f"token id: {token_id} | \
the token: {tokenizer.decoder.get(token_id)}")
embedding = token_emb_layer(torch.tensor(token_id, device=torch_device))
print(f"embedding shape: {embedding.shape}")

print("\n\ntoken embedding for the whole prompt")
token_embeddings = token_emb_layer(text_input.input_ids.to(torch_device))
print(token_embeddings.shape)
# batch size 1, 77 tokens, 768 values for each
token_embeddings

Positional embeddings tell the model where in a sequence a token is. Much like the token embedding, this is a set of (optionally learnable) parameters. But now instead of dealing with ~50k tokens we just need one for each position (77 total):

In [ ]:
pos_emb_layer = text_encoder.text_model.embeddings.position_embedding
print("the positional embedding layer")
display(pos_emb_layer)

position_ids = text_encoder.text_model.embeddings.position_ids[:, :77]
position_embeddings = pos_emb_layer(position_ids)
print("\npositional embedding for each position")
print(position_embeddings.shape)
display(position_embeddings)

In [ ]:
input_embeddings = token_embeddings + position_embeddings
input_embeddings2 = text_encoder.text_model.embeddings(
    text_input.input_ids.to(torch_device)
)

print((input_embeddings == input_embeddings2).all())

In [ ]:
from transformers.modeling_attn_mask_utils import \
_create_4d_causal_attention_mask

# start with these two to understand the logic of this method
  # ??text_encoder.forward # basically a wrapper
  # ??text_encoder.text_model.forward
def get_output_embeds(input_embeddings):
  # input_embeddings is the output of text_encoder.text_model.embeddings

  bsz, seq_len = input_embeddings.shape[:2]
  # TODO: why does CLIP's text model use causal mask?
  # ??_create_4d_causal_attention_mask
  causal_attention_mask = _create_4d_causal_attention_mask(
      input_shape = (bsz, seq_len), # batch_size * query_len
      dtype = input_embeddings.dtype,
      device = input_embeddings.device,
  )

  # ??text_encoder.text_model.encoder.forward
  encoder_outputs = text_encoder.text_model.encoder(
      inputs_embeds=input_embeddings,
      attention_mask=None,
      # We aren't using an attention mask so that can be None
      causal_attention_mask=causal_attention_mask.to(torch_device),
      output_attentions=None,
      output_hidden_states=True,
      # We want the output embs not the final(pooled) output
  )
  output = encoder_outputs.last_hidden_state

  # There is a final layer norm we need to pass these through
  output = text_encoder.text_model.final_layer_norm(output)
  return output

out_embs_test = get_output_embeds(input_embeddings)
(out_embs_test == output_embeddings).all()

Now that we have this process in place, we can replace the input embedding of a token with a new one of our choice - which in our final use-case will be something we learn. To demonstrate the concept though, let's replace the input embedding for 'puppy' in the prompt we've been playing with with the embedding for token 2368, get a new set of output embeddings based on this, and use these to generate an image to see what we get:

In [ ]:
text_encoder.get_input_embeddings()

In [ ]:
prompt = 'A picture of a puppy'

# Tokenize
text_input = tokenizer(
    prompt,
    padding="max_length",
    max_length=tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
)
input_ids = text_input.input_ids.to(torch_device)

# Get token embeddings
token_embeddings = token_emb_layer(input_ids)

# The new embedding.
# In this case just the input embedding of token 2368, which is a cat
replacement_token_embedding = token_emb_layer(
    torch.tensor(2368, device=torch_device)
)

# Insert this into the token embeddings (
token_embeddings[0, torch.where(input_ids[0]==6829)] =\
    replacement_token_embedding.to(torch_device)

input_embeddings = token_embeddings + position_embeddings
modified_output_embeddings = get_output_embeds(input_embeddings)

In [ ]:
def generate_with_embs(text_embeddings):

    # default height of Stable Diffusion
    height = 512
    # default width of Stable Diffusion
    width = 512
    # Number of denoising steps
    num_inference_steps = 30
    # Scale for classifier-free guidance
    guidance_scale = 7.5
    # Seed generator to create the inital latent noise
    generator = torch.manual_seed(32)
    batch_size = 1

    uncond_input = tokenizer(
      [""] * batch_size,
      padding="max_length",
      max_length=text_input.input_ids.shape[-1],
      return_tensors="pt"
    )
    with torch.no_grad():
        uncond_embeddings = text_encoder(
            uncond_input.input_ids.to(torch_device)
        ).last_hidden_state
    text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

    # Prep Scheduler
    set_timesteps(scheduler, num_inference_steps)

    # Prep latents
    latents = torch.randn(
    (batch_size, unet.config.in_channels, height // 8, width // 8),
    generator=generator,
    )
    latents = latents.to(torch_device)
    latents = latents * scheduler.init_noise_sigma

    # Loop
    for i, t in tqdm(
        enumerate(scheduler.timesteps),
        total=len(scheduler.timesteps)
    ):
        latent_model_input = torch.cat([latents] * 2)
        sigma = scheduler.sigmas[i]
        latent_model_input = scheduler\
                .scale_model_input(latent_model_input, t)

        # predict the noise residual
        with torch.no_grad():
            noise_pred = unet(
                latent_model_input,
                timestep = t,
                encoder_hidden_states=text_embeddings
            )["sample"]

        # perform guidance
        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        noise_pred = (
            noise_pred_uncond
            +guidance_scale * (noise_pred_text - noise_pred_uncond)
        )

        # compute the previous noisy sample x_t -> x_t-1
        latents = scheduler.step(noise_pred, t, latents).prev_sample

    return latents_to_pil(latents)[0]


In [ ]:
generate_with_embs(modified_output_embeddings)

**What can we do with this?** once we can access and modify the token embeddings we can do tricks like
- replacing them with something else. In the example we just did, that was just another token embedding from the model's vocabulary, equivalent to just editing the prompt.
- we can also mix tokens - for example, here's a half-puppy-half-skunk:

In [ ]:
# In case you're wondering how to get the token for a word,
# or the embedding for a token:
prompt = 'skunk'
print('tokenizer(prompt):', tokenizer(prompt))
print('token_emb_layer([token_id]) shape:',
      token_emb_layer(torch.tensor([8797], device=torch_device)).shape)

In [ ]:
# Get the initial token embeddings
token_embeddings = token_emb_layer(input_ids)

# The new embedding. Which is now a mixture of
# the token embeddings for 'puppy' and 'skunk'
puppy_token_embedding = token_emb_layer(
  torch.tensor(6829, device=torch_device)
)
skunk_token_embedding = token_emb_layer(
  torch.tensor(42194, device=torch_device)
)
replacement_token_embedding = (0.5*puppy_token_embedding+
                               0.5*skunk_token_embedding)

# Insert this into the token embeddings (
token_embeddings[0, torch.where(input_ids[0]==6829)]=\
  replacement_token_embedding.to(torch_device)

input_embeddings = token_embeddings + position_embeddings
modified_output_embeddings = get_output_embeds(input_embeddings)
generate_with_embs(modified_output_embeddings)

### Textual Inversion

OK, so we can slip in a modified token embedding, and use this to generate an image. We used the token embedding for 'cat' in the above example, but what if instead could 'learn' a new token embedding for a specific concept? This is the idea behind 'Textual Inversion', in which a few example images are used to create a new token embedding.

[textual inversion blog post](https://textual-inversion.github.io/static/images/training/training.JPG) - note it doesn't show the positional embeddings step for simplicity_

We won't cover how this training works, but we can try loading one of these new 'concepts' from the [community-created SD concepts library](https://huggingface.co/sd-concepts-library) and see how it fits in with our example above.

In [ ]:
# main_path = "https://huggingface.co/sd-concepts-library"
# url = f"{main_path}/birb-style/resolve/main/learned_embeds.bin"
# !wget {url}

In [ ]:
birb_embed = torch.load('learned_embeds.bin')
birb_embed.keys(), birb_embed['<birb-style>'].shape

We get a dictionary with a key (the special placeholder I used, <birb-style>) and the corresponding token embedding. As in the previous example, let's replace the 'puppy' token embedding with this and see what happens:

In [ ]:
prompt = 'A mouse in the style of puppy'

# Tokenize
text_input = tokenizer(
    prompt, padding="max_length", max_length=tokenizer.model_max_length,
    truncation=True, return_tensors="pt"
)
input_ids = text_input.input_ids.to(torch_device)
token_embeddings = token_emb_layer(input_ids)
replacement_token_embedding = birb_embed['<birb-style>'].to(torch_device)
# replace "puppy" token
token_embeddings[0, torch.where(input_ids[0]==6829)]=\
  replacement_token_embedding.to(torch_device)
input_embeddings = token_embeddings + position_embeddings
modified_output_embeddings = get_output_embeds(input_embeddings)
generate_with_embs(modified_output_embeddings)

Again, there is [a nice inference notebook ](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_conceptualizer_inference.ipynb) from hf to make it easy to use the different concepts, that properly handles using the names in prompts ("A \<cat-toy> in the style of \<birb-style>") without worrying about all this manual stuff. The goal of this notebook is to pull back the curtain a bit so you know what is going on behind the scenes :)

### Messing with Embeddings

Besides just replacing the token embedding of a single word, there are various other tricks we can try. For example, what if we create a 'chimera' by averaging the embeddings of two different prompts?

In [ ]:
text_input1 = tokenizer(
  ["A mouse"], padding="max_length",
  max_length=tokenizer.model_max_length,
  truncation=True, return_tensors="pt"
)
text_input2 = tokenizer(
  ["A leopard"], padding="max_length",
  max_length=tokenizer.model_max_length,
  truncation=True, return_tensors="pt"
)
with torch.no_grad():
    text_embeddings1 = text_encoder(
      text_input1.input_ids.to(torch_device)
    )[0]
    text_embeddings2 = text_encoder(
      text_input2.input_ids.to(torch_device)
    )[0]

# Mix them together
mix_factor = 0.35
mixed_embeddings = (text_embeddings1*mix_factor + \
                   text_embeddings2*(1-mix_factor))
generate_with_embs(mixed_embeddings)

## The UNET

Now it's time we looked at the actual diffusion model. This is typically a Unet that takes in the noisy latents (x) and predicts the noise. We use a conditional model that also takes in the timestep (t) and our text embedding (aka encoder_hidden_states) as conditioning. Feeding all of these into the model looks like this:
`noise_pred = unet(latents, t, encoder_hidden_states=text_embeddings)["sample"]`


Given a set of noisy latents, the model predicts the noise component. We can remove this noise from the noisy latents to see what the output image looks like (`latents_x0 = latents - sigma * noise_pred`). And we can add most of the noise back to this predicted output to get the (slightly less noisy hopefully) input for the next diffusion step. To visualize this let's generate another image, saving both the predicted output (x0) and the next step (xt-1) after every step:

In [ ]:
prompt = 'Oil painting of an otter in a top hat'

# Make a folder to store results
!rm -rf steps/
!mkdir -p steps/

# Prep text
text_input = tokenizer(
    [prompt], padding="max_length",
    max_length=tokenizer.model_max_length,
    truncation=True, return_tensors="pt"
)
with torch.no_grad():
    text_embeddings = text_encoder(
        text_input.input_ids.to(torch_device)
    ).last_hidden_state
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size,
    padding="max_length",
    max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
    uncond_embeddings = text_encoder(
        uncond_input.input_ids.to(torch_device)
    ).last_hidden_state

text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

# Prep Scheduler
set_timesteps(scheduler, num_inference_steps)

# Prep latents
latents = torch.randn(
  (batch_size, unet.config.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents.to(torch_device)
latents = latents * scheduler.init_noise_sigma

# Loop
for i, t in tqdm(
    enumerate(scheduler.timesteps),
    total=len(scheduler.timesteps)
):
    latent_model_input = torch.cat([latents] * 2)
    sigma = scheduler.sigmas[i]
    latent_model_input = scheduler\
      .scale_model_input(latent_model_input, t)

    with torch.no_grad():
        noise_pred = unet(
            latent_model_input, t, encoder_hidden_states=text_embeddings
        )["sample"]

    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = (
        noise_pred_uncond
        +guidance_scale * (noise_pred_text - noise_pred_uncond)
    )

    # Get the predicted x0:
    # latents_x0 = latents - sigma * noise_pred # Calculating ourselves
    scheduler_step = scheduler.step(noise_pred, t, latents)
    # Using the scheduler (Diffusers 0.4 and above)
    latents_x0 = scheduler_step.pred_original_sample

    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler_step.prev_sample

    # To PIL Images
    im_t0 = latents_to_pil(latents_x0)[0]
    im_next = latents_to_pil(latents)[0]

    # Combine the two images and save for later viewing
    # The version on the right shows the predicted 'final output'
    # (x0) at each step.
    im = Image.new('RGB', (1024, 512))
    im.paste(im_next, (0, 0))
    im.paste(im_t0, (512, 0))
    im.save(f'steps/{i:04}.jpeg')

In [ ]:
# Make and show the progress video (change width to 1024 for full res)
!ffmpeg -v 1 -y -f image2 -framerate 12 -i steps/%04d.jpeg -c:v libx264 -preset slow -qp 18 -pix_fmt yuv420p out.mp4
mp4 = open('out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

### Classifier Free Guidance

By default, the model doesn't often do what we ask. If we want it to follow the prompt better, we use a hack called CFG. There's a good explanation in this video (AI coffee break GLIDE).

In the code, this comes down to us doing:

`noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)`

This works suprisingly well :) Explore changing the guidance_scale in the code above and see how this affects the results. How high can you push it before the results get worse?

### Sampling

There is still some complexity hidden from us inside `latents = scheduler.step(noise_pred, i, latents)["prev_sample"]`. How exactly does the sampler go from the current noisy latents to a slightly less noisy version? Why don't we just use the model in a single step? Are there other ways to view this?

The model tries to predict the noise in an image. For low noise values, we assume it does a pretty good job. For higher noise levels, it has a hard task! So instead of producing a perfect image, the results tend to look like a blurry mess - see the start of the video above for a visual! So, samplers use the model predictions to move a small amount towards the model prediction (removing some of the noise) and then get another prediction based on this marginally-less-rubbish input, and hope that this iteratively improves the result.

Different samplers do this in different ways. You can try to inspect the code for the default LMS sampler with:

In [ ]:
# ??scheduler.step

**Time to draw some diagrams!** (Whiteboard/paper interlude)

## Guidance

OK, final trick! How can we add some extra control to this generation process?

At each step, we're going to use our model as before to predict the noise component of x. Then we'll use this to produce a predicted output image, and apply some loss function to this image.

This function can be anything, but let's demo with a super simple example. If we want images that have a lot of blue, we can craft a loss function that gives a high loss if pixels have a low blue component:

In [ ]:
def blue_loss(images):
    # How far are the blue channel values to 0.9:
    error = torch.abs(images[:,2] - 0.9).mean()
    # [:,2] -> all images in batch, only the blue channel
    return error

During each update step, we find the gradient of the loss with respect to the current noisy latents, and tweak them in the direction that reduces this loss as well as performing the normal update step:

In [ ]:
prompt = 'A campfire (oil on canvas)'#@param
# default height of Stable Diffusion
height = 512
# default width of Stable Diffusion
width = 512
# Number of denoising steps
num_inference_steps = 50  #@param
# Scale for classifier-free guidance
guidance_scale = 8 #@param
# Seed generator to create the inital latent noise
generator = torch.manual_seed(32)
batch_size = 1
blue_loss_scale = 200 #@param

# Prep text
text_input = tokenizer(
    [prompt], padding="max_length",
    max_length=tokenizer.model_max_length,
    truncation=True, return_tensors="pt"
)
with torch.no_grad():
    text_embeddings = text_encoder(
        text_input.input_ids.to(torch_device)
    ).last_hidden_state
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size,
    padding="max_length",
    max_length=max_length,
    return_tensors="pt"
)
with torch.no_grad():
    uncond_embeddings = text_encoder(
        uncond_input.input_ids.to(torch_device)
    ).last_hidden_state
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

set_timesteps(scheduler, num_inference_steps)
latents = torch.randn(
  (batch_size, unet.config.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents.to(torch_device)
latents = latents * scheduler.init_noise_sigma

for i, t in tqdm(
    enumerate(scheduler.timesteps),
    total=len(scheduler.timesteps)
):
    latent_model_input = torch.cat([latents] * 2)
    sigma = scheduler.sigmas[i]
    latent_model_input = scheduler\
      .scale_model_input(latent_model_input, t)

    # predict the noise residual
    with torch.no_grad():
        noise_pred = unet(
            latent_model_input,
            timestep = t,
            encoder_hidden_states=text_embeddings
        )["sample"]

    # perform CFG
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = (
        noise_pred_uncond
        +guidance_scale * (noise_pred_text - noise_pred_uncond)
    )

    #### ADDITIONAL GUIDANCE ###
    if i%5 == 0:
        latents = latents.detach().requires_grad_()
        # This gives you gradients only w.r.t. the current latents,
        # not the entire generation history.
        # If you did not detach the latents before requires_grad_():
          # 1.The graph would include all previous UNet forward passes
          # 2.Backprop would attempt to compute gradients through
          # the entire diffusion history
          # 3.UNet weights (although wrapped in no_grad) and scheduler ops
          # would be included incorrectly
          # 4.You'd get massive memory usage and likely crashes
        latents_x0 = latents - sigma * noise_pred
        # latents_x0 = scheduler\
        #   .step(noise_pred, t, latents).pred_original_sample

        # Decode to image space
        denoised_images = vae\
          .decode(
              (1 / 0.18215) * latents_x0
          ).sample / 2 + 0.5 # range (0, 1)

        # Calculate loss
        loss = blue_loss(denoised_images) * blue_loss_scale

        # Occasionally print it out
        if i%10==0:
            print(i, 'loss:', loss.item())

        # Get gradient
        cond_grad = torch.autograd.grad(loss, latents)[0]

        # Modify the latents based on this gradient
        latents = latents.detach() - cond_grad * sigma**2
        # This keeps the diffusion loop cheap and stateless
        # from the perspective of autograd.
        # Without detach(), that new latent would still carry:
        #   the gradient relationship to the grad
        #   the gradient relationship to the loss
        #   the gradient relationship to the VAE decode

    # Now step with scheduler
    latents = scheduler.step(noise_pred, t, latents).prev_sample


latents_to_pil(latents)[0]

NB: We should set latents requires_grad=True **before** we do the forward pass of the unet (removing `with torch.no_grad()`) if we want mode accurate gradients. BUT this requires a lot of extra memory. You'll see both approaches used depending on whose implementation you're looking at.